In [ ]:
from matplotlib.pylab import *
from matplotlib import pylab
import numpy as np
import math

import Hankel
import LTI_systems as LTI
import n_masses_springs_dampers_system
from estimate_ss_all import estimate_ss

In [ ]:
nx = 6

matrix_type = 2
    # 1: matrix A_c diagonalizable 
    # 2: matrix A_c non diagonalizable
    # 3: matrix A_c from masses-springs-dampers system

complesso_coniugati = 0
if matrix_type == 3: complesso_coniugati = 1  # serve per attivare un algoritmo, TODO: attivarlo riconoscendo la presenza di autovalori complesso-coniugati

basis_change = 1 # 0=A_c is in modal coordinates
T_orthogonal = 1 
print("matrix_type = ",matrix_type," , basis_change = ",basis_change," , T_orthogonal = ",T_orthogonal)

Auns = np.triu(np.random.randn(nx,nx), 1)

T = np.array(np.random.randn(nx,nx))
if T_orthogonal:
    T,_ = np.linalg.qr(T)  
#endif
#print("T = ",T)

matrix_type_3_parameters = []

if matrix_type == 1 or matrix_type == 2:
    Tsc = 0.04
elif matrix_type == 3:
    Tsc = 0.0001
#endif

def create_Ac(theta=np.array([None]), verbose=False):
    global matrix_type_3_parameters

    if matrix_type == 1 or matrix_type == 2: 
        Ac = np.diag(theta)
        if matrix_type == 2: 
            c_offdiag = 0.1
            if 1:
                beta_uns = c_offdiag * np.abs(theta)
                Ac = Ac + np.diag(beta_uns) @ Auns
            else:
                Ac = Ac + beta_uns * Auns
            #endif
        #endif
        if basis_change:
            Ac = np.asmatrix( np.linalg.inv(T) @ Ac @ T)
            if 0: plt.figure(); plt.plot(T[:,0]); plt.show()            
        #endif
    elif matrix_type == 3:
        ndof = int(nx / 2)
        tmpstd = 1.e-0
        parameters = {}
        parameters['M'] = 27.0 * (1 + 0.01*tmpstd*np.random.randn(ndof))
        parameters['K'] = 1.0e5 * (1 + 0.1*tmpstd*np.random.randn(ndof))
        parameters['C'] = 1.0e4 * (1 + 0.1*tmpstd*np.random.randn(ndof))
        parameters['deltaX'] = np.linspace(0.0, 1.0, ndof)
        #print("parameters['deltaX'] = ",parameters['deltaX'])
        if theta.any() == None and len(matrix_type_3_parameters)==0: 
            matrix_type_3_parameters = parameters
        else:
            parameters = matrix_type_3_parameters
        #endif
        print("matrix_type_3_parameters = ",matrix_type_3_parameters)
        print("parameters = ",parameters)
        Ac,Bc,Cc,Dc = n_masses_springs_dampers_system.build(4,parameters)
        if 0:
            tmptmp,tmpeigv = np.linalg.eig(Ac)
            for jj in range(int(nx/2)):
                plt.figure(); plt.plot(tmpeigv[:,jj]); 
            #endfor
            plt.show()
        #endif
    #endif
    return Ac

In [ ]:
def build_model_and_generate_data(nx, theta, verbose=False):
    N = 230
    Nt = 500

    B = np.asmatrix(np.ones(nx)).T
    p = 2
    C = np.atleast_2d(np.zeros((p,nx)));
    C[0:p,0:p] = np.eye(p)
    D = [0.0]
    x0 = np.asmatrix(np.zeros(nx)).T
    
    if matrix_type == 3:
        Ac = create_Ac()
    else:
        Ac = create_Ac(theta)
    #endif
    print("Ac_vera = ",Ac)
    Ac_eigs = np.linalg.eigvals(Ac)

    A = np.linalg.inv(np.eye(Ac.shape[0]) - Tsc*Ac)
    #print("A = ",A)
    B = Tsc * A @ B
    A_orig = A.copy(); B_orig = B.copy(); C_orig = C.copy() 
    
    A_eigs = np.linalg.eigvals(A)
    #print("A = ",A)
       
    #
    std_u = 0.0 #1.e0
    if False:
        u = np.arange(0, Nt);
    else:
        if True:
            u = np.atleast_2d(np.zeros(Nt)); 
            u[:,0:int(Nt/4)] = 100.0
            u = np.asmatrix(u)
            
        if False:  
            u = np.random.randn(1, Nt)
            u = np.asmatrix(u)
        
        if False:
            u = np.atleast_2d(np.zeros(Nt));
            u[:,5:10] = 30.0
            u[:,20:40] = 40.0
            u[:,80:110] = 80.0
            u[:,140:170] = 40.0
            u[:,200:210] = 100.0
            u = np.asmatrix(u)
        #u = u + 1.e1 * np.random.randn(u.shape[0], Nt)    # add noise
        
        u_meas = u[:,0:N] + std_u * np.random.randn(u.shape[0], N)
    #endif

    #print("A.shape = ",A.shape)
    y, X_hist = LTI.simula_DLTI_StateSpace(A, B, C, D, u, x0)
    #print(X_hist.shape)
    tmpU,tmpS,tmpV = np.linalg.svd(X_hist,full_matrices=True); tmpV=tmpV.T; 
    if verbose: print("singular values of X_hist = ",tmpS)
    Xlastrow = X_hist[-1,:]
    #print("Xlastrow.shape = ",Xlastrow.shape)
    if verbose: print("max(|X_hist[-1,:]|) = ",np.max(np.abs(Xlastrow)))
    if verbose: print("||X_hist[-1,:]||_2 = ",np.linalg.norm(Xlastrow))
    Alastcol = np.atleast_2d(A[0:nx,-1])
    if np.diff(Alastcol.shape)>0: Alastcol = Alastcol.T
    #print("Alastcol.shape = ",Alastcol.shape)
    if verbose: print("max(|A[0:nx,-1] @ X_hist[-1,:]|) = ",np.max(np.abs(Alastcol @ Xlastrow)))
    if verbose: print("||A[0:nx,-1] @ X_hist[-1,:]||_2) = ",np.linalg.norm(Alastcol @ Xlastrow))
    # add measurement noise:
    if nx == 7:
        std_y = 0.5e1
    else:
        std_y = 1.0 * 0.05* np.amax(np.abs(y))  #1.0 * 0.03* np.amax(np.abs(y)) 
    #endif
    std_y = 0.0
    y_meas = y[:,0:N] + std_y * np.random.randn(y.shape[0], N)

    t = np.asmatrix(range(1000))

    # Number of inputs and outputs
    m = u.shape[0]
    p = y.shape[0]

    # Time index
    t = np.asmatrix(range(Nt))

    if verbose: 
        pylab.figure(1)
        for ii in range(m):
            pylab.subplot(m,1,ii+1);
            pylab.plot(np.squeeze(np.asarray(u_meas[ii, :])), 'r-'); 
            pylab.plot(np.squeeze(np.asarray(u[ii,:])), 'b:'); 
            pylab.grid();
            pylab.ylabel('Input u'+str(ii+1))
        pylab.show()

        pylab.figure(2)
        for ii in range(p):
            pylab.subplot(p,1,ii+1);
            pylab.plot(np.squeeze(np.asarray(y_meas[ii, :])), 'r-'); 
            pylab.plot(np.squeeze(np.asarray(y[ii,:])), 'b:'); 
            pylab.grid();
            pylab.ylabel('Output y'+str(ii+1))
        pylab.show()
    #endif
    return u, y, y_meas, u_meas, N, Nt,std_y,std_u, A_eigs, A_orig, B_orig, C_orig, D, x0, X_hist

In [ ]:
alltheta = np.array([ -0.9, -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.15, -0.1, -0.05, -0.01])
tmpr = len(alltheta)/nx
I = np.round(np.arange(0,nx)*tmpr)
if matrix_type < 3:
    theta = alltheta[I.astype(int)]
else:
    theta = alltheta.copy()
#endif
print(theta)
u, y, y_meas, u_meas, N, Nt,std_y,std_u, A_eigs, A_vera, B_vera, C, D, x0, X_hist = build_model_and_generate_data(nx,theta)
ny = C.shape[0]

In [ ]:
plt.figure()
plt.plot(u[0,0:y_meas.shape[1]].T)

plt.figure()
plt.plot(y_meas.T)
plt.show()
u_hist, s_hist, v_hist = np.linalg.svd(X_hist)
print(s_hist)

In [ ]:
def estimate_model(itest, metodo_def_M=1, theta_est=[], verbose=False, delete_permutations = False):
    #TODO: "A_vera","Tsc","X_hist","nx" da variabile globale deve essere passata alla funzione.
    # NB: the implementation of the subspace method has been extracted from the accompanying software of:
    # Zhang Liu and Lieven Vandenberghe. "Interior-point method for nuclear norm approximation with application to system identification"
    # and precisely from the source file "sysid.py"
    
    
    D_A_vera = np.linalg.eigvals(A_vera)
    if verbose: print("A_vera eigenvalues = ", D_A_vera)
    
    m = u.shape[0]
    p = y.shape[0]

    # r = block row dimension in Hankel matrices Y and U
    r = int(nx/p)*4+nx #*5 #*10 #20

    # Dimensions of block Hankel matrices, Y: r*p x b, U: r*m x b,
    a1 = r * p
    c = r * m
    b = N - r + 1

    # Construct Hankel matrices
    Y = Hankel.matrix(y_meas, r, b, p=p, q=1)
    U = Hankel.matrix(u_meas, r, b, p=m, q=1)
    
    # Form the projection matrix
    Uperp = np.asmatrix(np.identity(b)) - (U.T) @ np.linalg.solve(U @ U.T, U) 

    # Compute the matrix G
    G = Y @ Uperp

    U1, svN, V1 = np.linalg.svd(G.copy(), full_matrices = False)
    
    nxo = nx
    Obs = np.asmatrix(U1[:,:nxo])

    for ii in range(nxo):
        if 1:
            Obs[:,ii] *= np.sqrt(svN[ii])
        else: # see Vandenberghe
            Obs[:,ii] *= svN[ii]
        #endif
    #endfor
    
    Ac_est_eigs = np.linalg.eigvals(Ac_est)
    if verbose: print("Ac_est eigenvalues = ",Ac_est_eigs)
            
    do_all_possible_permutations = 0
    
    ssm = estimate_ss(Obs, y_meas, u_meas, Tsc, C_f = C, A_vera=A_vera, A_vera_est=A_vera_est, metodo_def_M=metodo_def_M, \
                  matrix_type=matrix_type, verbose=verbose, permutazioni_su_tutte_le_variabili = do_all_possible_permutations, delete_permutations = delete_permutations)   
    A_o = np.asmatrix(ssm['A'])
    B_o = np.asmatrix(ssm['B'])
    C_o = np.asmatrix(ssm['C'])
    D_o = np.asmatrix(ssm['D'])
    x0_o = np.asmatrix(ssm['x0'])

    As = np.asmatrix(ssm['As'])
    Bs = np.asmatrix(ssm['Bs'])
    Cs = np.asmatrix(ssm['Cs'])
    Ds = np.asmatrix(ssm['Ds'])
    x0s = np.asmatrix(ssm['x0s'])
    ys, Xs_hist = LTI.simula_DLTI_StateSpace(As, Bs, Cs, Ds, u, x0s)

    if 'tmpMm_at_min' in ssm: tmpMm_at_min[iTest] = ssm['tmpMm_at_min']
    if 'tmpMu_at_min' in ssm: tmpMu_at_min[iTest] = ssm['tmpMu_at_min']
    if 'tmpMu_hist' in ssm and 'tmpMm_hist' in ssm: 
        tmpMu_hist[iTest] = np.var(ssm['tmpMu_hist'])
        if 1:
            tmp = np.array(ssm['tmpMu_hist'])
            tmp2 = np.array(ssm['tmpMm_hist'])
            tmpx = np.array(ssm['tmp_2err_Ac_est_sub_hist'])
            print("tmp.shape = ",tmp.shape)
            plt.figure(); 
            plt.plot(tmpx,tmp,'bo',label='tmpMu_hist'); 
            plt.plot(tmpx,tmp2,'r*',label='tmpMm_hist'); 
            plt.show()
        #endif
    #endif
    if 'relerr_unmeas' in ssm: relerr_unmeas[iTest] = ssm['relerr_unmeas']
    if 'max_Tx_col_norms' in ssm: max_Tx_col_norms[iTest] = ssm['max_Tx_col_norms']
    if 'check_ortog_Tx' in ssm: check_ortog_Tx[iTest] = ssm['check_ortog_Tx']
    if 'cond_M' in ssm: cond_M[iTest] = ssm['cond_M']
    if 'cond_Tx' in ssm: cond_Tx[iTest] = ssm['cond_Tx']

    if 'maxrelerr_eigenvalues_A' in ssm: maxrelerr_eigenvalues_A[iTest] = ssm['maxrelerr_eigenvalues_A']
    
    if 'relerr_vero_Af_ps_00' in ssm: relerr_vero_Af_ps_00[iTest] = ssm['relerr_vero_Af_ps_00']
    if 'relerr_vero_Af_perm_00' in ssm: relerr_vero_Af_perm_00[iTest] = ssm['relerr_vero_Af_perm_00']
    if 'maxrelerr_vero_Af_ps_sub' in ssm: maxrelerr_vero_Af_ps_sub[iTest] = ssm['maxrelerr_vero_Af_ps_sub']
    if 'maxrelerr_vero_Af_perm_sub' in ssm: maxrelerr_vero_Af_perm_sub[iTest] = ssm['maxrelerr_vero_Af_perm_sub']
    if 'maxrelerr_vero_Af_ps_alldiag' in ssm: maxrelerr_vero_Af_ps_alldiag[iTest] = ssm['maxrelerr_vero_Af_ps_alldiag']
    if 'maxscalerr_vero_Af_ps_all' in ssm: maxscalerr_vero_Af_ps_all[iTest] = ssm['maxscalerr_vero_Af_ps_all']
    if 'maxscalerr_vero_Af_perm_all' in ssm: maxscalerr_vero_Af_perm_all[iTest] = ssm['maxscalerr_vero_Af_perm_all']
    
    if 'maxrelerr_vero_Ac_ps_sub' in ssm: maxrelerr_vero_Ac_ps_sub[iTest] = ssm['maxrelerr_vero_Ac_ps_sub']
    if 'maxrelerr_vero_Ac_no_perm' in ssm: maxrelerr_vero_Ac_no_perm[iTest] = ssm['maxrelerr_vero_Ac_no_perm']
    if 'maxrelerr_vero_Ac_perm_sub' in ssm: maxrelerr_vero_Ac_perm_sub[iTest] = ssm['maxrelerr_vero_Ac_perm_sub']
    if 'minrelerr_vero_Ac_ps_sub' in ssm: minrelerr_vero_Ac_ps_sub[iTest] = ssm['minrelerr_vero_Ac_ps_sub']
    if 'minrelerr_vero_Ac_no_perm' in ssm: minrelerr_vero_Ac_no_perm[iTest] = ssm['minrelerr_vero_Ac_no_perm']
    if 'minrelerr_vero_Ac_perm_sub' in ssm: minrelerr_vero_Ac_perm_sub[iTest] = ssm['minrelerr_vero_Ac_perm_sub']
    if 'min_relerr_A11' in ssm: relerr_A11[iTest] = ssm['min_relerr_A11']
    relerr_Asub[iTest] = ssm['min_relerr_est_A11']
    if 'relerr_Anp11' in ssm: relerr_Anp11[iTest] = ssm['relerr_Anp11']
    relerr_Asps11[iTest] = ssm['relerr_Asps11']
    
    y_o, X_o_hist = LTI.simula_DLTI_StateSpace(A_o, B_o, C_o, D_o, u, x0_o)
    Y_o = Hankel.matrix(y_o[:,0:N], r, b, p=p, q=1)
    G_o = Y_o @ Uperp
    if verbose: print('nuclear norm of G_o:', np.linalg.norm(G_o, 'nuc'))
    if verbose: print("Sistema identificato:")
    if verbose: print('A_o = ', A_o)
    if verbose: print()
    if verbose: print('B_o = ', B_o)
    if verbose: print()
    if verbose: print('C_o = ', C_o)
    if verbose: print()
    #print("A_o_eigs = ",np.linalg.eigvals(A_o)[-1::-1])
    D,V = np.linalg.eig(A_o)
    tmpU,tmpS,tmpV = np.linalg.svd(A_o); tmpV=tmpV.T;
    if verbose: print("A_o eigenvalues = ",D)
    if verbose: print("errore relativo stima coefficienti A = ",np.abs(A_vera - A_o)/np.abs(A_vera))
    if not complesso_coniugati and matrix_type <= 2: 
        relerr_eigvals = np.zeros(nx)
        errang_eigvect = np.zeros(nx)
        for i in range(nx):
            if verbose: print(D[i]-D_A_vera[i])
            relerr_eigvals[i] = np.abs(D[i]-D_A_vera[i])/np.abs(D_A_vera[i])
            #errang_eigvect[i] = (V[:,i].T @ V_A_vera[:,i])/(np.linalg.norm(V[:,i])*np.linalg.norm(V_A_vera[:,i]))
        #endfor
        if verbose: print("errore relativo autovalori = ",relerr_eigvals)
        if verbose: print("angolo errore autovettori = ",errang_eigvect)
    #endif
    if verbose: print('nuclear norm of A_o:', np.linalg.norm(A_o, 'nuc'))
    if verbose: print("true system:")
    if verbose: print('A_vera = ', A_vera)
    if verbose: print()
    if verbose: print('B_vera = ', B_vera)
    if verbose: print()
    if verbose: print('C_vera = ', C)
    if verbose: print()
    if verbose: print('nuclear norm of A_vera:', np.linalg.norm(A_vera, 'nuc'))
    if verbose: 
        plt.figure(); plt.title("state 0 trajectory")
        plt.plot(X_o_hist[0,:].T,'r-',label='estimated'); 
        plt.plot(Xs_hist[0,:].T,'k:',label='subspace'); 
        plt.plot(X_hist[0,:].T,'b--',label='true'); 
        plt.legend()

        plt.figure(); plt.title("state 1 trajectory")
        plt.plot(X_o_hist[1,:].T,'r-',label='estimated'); 
        plt.plot(Xs_hist[1,:].T,'k:',label='subspace'); 
        plt.plot(X_hist[1,:].T,'b--',label='true'); 
        plt.legend()

        plt.figure(); plt.title("state 2 trajectory")
        plt.plot(X_o_hist[2,:].T,'r-',label='estimated'); 
        plt.plot(Xs_hist[2,:].T,'k:',label='subspace'); 
        plt.plot(X_hist[2,:].T,'b--',label='true'); 
        plt.legend()
    #endif
    ye_o = np.asmatrix(np.zeros((p,Nt)))
    xi = x0_o
    for ii in range(Nt):
        ye_o[:,ii] = C_o * xi + D_o * u[:,ii]
        xi = A_o * xi + B_o * u[:,ii]
    #endfor
    plt.show()
    return A_o

In [ ]:
if matrix_type == 3:
    theta_est = 0.0
    Ac_est = create_Ac()
else:
    theta_est = theta + 0.0 * np.random.randn(len(theta))
    Ac_est = create_Ac(theta_est)
#endif
print("Ac_est[0:",ny,",0:",ny,"] = ",Ac_est[0:ny,0:ny])
# discretizzo la matrice:
A_vera_est = np.linalg.inv(np.eye(Ac_est.shape[0]) - Tsc*Ac_est)
generate_data = True

In [ ]:
Ntest = 1
std_initial_est_err = 0.0

tmpMm_at_min = np.zeros(Ntest)
tmpMu_at_min = np.zeros(Ntest)
tmp_2err_Ac_est_sub_hist = np.zeros(Ntest)
tmpMm_hist = np.zeros(Ntest)
tmpMu_hist = np.zeros(Ntest)
relerr_unmeas = np.zeros(Ntest)
max_Tx_col_norms = np.zeros(Ntest)
min_cosine_Tx = np.zeros(Ntest)
colsum_cosine_Tx = np.zeros(Ntest)
check_ortog_Tx = np.zeros(Ntest)
cond_M = np.zeros(Ntest)
cond_Tx = np.zeros(Ntest)
maxrelerr_eigenvalues_A = np.zeros(Ntest)
relerr_vero_Af_ps_00 = np.zeros(Ntest)
relerr_vero_Af_perm_00 = np.zeros(Ntest)
maxrelerr_vero_Af_ps_sub = np.zeros(Ntest)
maxrelerr_vero_Af_perm_sub = np.zeros(Ntest)
maxrelerr_vero_Ac_ps_sub = np.zeros(Ntest)
maxrelerr_vero_Ac_no_perm = np.zeros(Ntest)
maxrelerr_vero_Ac_perm_sub = np.zeros(Ntest)
minrelerr_vero_Ac_ps_sub = np.zeros(Ntest)
minrelerr_vero_Ac_no_perm = np.zeros(Ntest)
minrelerr_vero_Ac_perm_sub = np.zeros(Ntest)
maxrelerr_vero_Af_ps_alldiag = np.zeros(Ntest)
maxscalerr_vero_Af_ps_all = np.zeros(Ntest)
maxscalerr_vero_Af_perm_all = np.zeros(Ntest)
relerr_Asub = np.zeros(Ntest)
relerr_A11 = np.zeros(Ntest)
relerr_Ac11 = np.zeros(Ntest); relerr_Ac12 = np.zeros(Ntest); relerr_Ac21 = np.zeros(Ntest); relerr_Ac22 = np.zeros(Ntest)
relerr_est_Ac11 = np.zeros(Ntest); relerr_est_Ac12 = np.zeros(Ntest); relerr_est_Ac21 = np.zeros(Ntest); relerr_est_Ac22 = np.zeros(Ntest)
relerr_Anp11 = np.zeros(Ntest)
relerr_Asps11 = np.zeros(Ntest)
usato_autovett_non_mis = np.zeros(Ntest)
errore_A11_est = np.zeros(Ntest)

for iTest in range(Ntest):
    print("iTest = ",iTest)
    if matrix_type == 3:
        theta = np.array([None])
    #endif
    if iTest >= 0:
        T_creaA = np.array(np.random.randn(nx,nx))
        if T_orthogonal:
            T_creaA,_ = np.linalg.qr(T_creaA)
        #endif    
        if matrix_type == 3:
            theta_est = np.array([0.0])
            Ac_est = create_Ac()
        else:
            theta_est = theta * (1.0 + std_initial_est_err * np.random.randn(len(theta)))
            Ac_est = create_Ac(theta_est) 
        #endif
    #endif
    A_vera_est = np.linalg.inv(np.eye(Ac_est.shape[0]) - Tsc*Ac_est)
    
    Ac_vera_cont = create_Ac(theta)
    
    # N.B.: la riduzione del numero di permutazioni si può applicare solo se metodo_def_M = 2
    metodo_imposizione_Cf = 2
    
    if Ntest == 1:
        if generate_data:
            u, y, y_meas, u_meas, N, Nt,std_y,std_u, A_eigs, A_vera, B_vera, C, D, x0, X_hist = \
                build_model_and_generate_data(nx, theta, verbose=True)    
        #endif
        A_o = estimate_model(iTest, metodo_def_M=metodo_imposizione_Cf, theta_est=theta_est, verbose=True, delete_permutations=True)
    else:
        if generate_data:
            u, y, y_meas, u_meas, N, Nt,std_y,std_u, A_eigs, A_vera, B_vera, C, D, x0, X_hist = build_model_and_generate_data(nx, theta)    
        #endif
        A_o = estimate_model(iTest, metodo_def_M=metodo_imposizione_Cf, theta_est=theta_est, verbose=False, delete_permutations=True)        
    #endif
    matrix_type_3_parameters = []
#endfor

In [ ]:
maxrelerr_eigenvalues_A

In [ ]:
maxrelerr_vero_Ac_ps_sub

In [ ]:
maxrelerr_vero_Ac_perm_sub

In [ ]:
ymax = 24
xmax = 2.0
vbins = np.arange(0,xmax+0.001,xmax/100)

plt.figure(); plt.grid(linestyle='-')
plt.hist(maxrelerr_vero_Ac_ps_sub,rwidth=None,color='r', range=(0.0,1.0), bins=vbins, cumulative=False, histtype='bar', align='left', label='standard ss') #,density=True)
plt.legend(loc=7); plt.xlim(-1.e-2,xmax); plt.ylim(0.0,ymax)

plt.figure(); plt.grid(linestyle='-')
plt.hist(maxrelerr_vero_Ac_perm_sub,rwidth=None,color='b', range=(0,1.0), bins=vbins, cumulative=False, histtype='bar', align='left', label='opt perm') #,density=True)
plt.legend(loc=7); plt.xlim(-1.e-2,xmax); plt.ylim(0.0,ymax)

plt.figure(); plt.grid(linestyle='-')
plt.hist(minrelerr_vero_Ac_ps_sub,rwidth=None,color='r', range=(0.0,1.0), bins=vbins, cumulative=False, histtype='bar', align='left', label='standard ss') #,density=True)
plt.legend(loc=7); plt.xlim(-1.e-2,xmax); plt.ylim(0.0,ymax)

plt.figure(); plt.grid(linestyle='-')
plt.hist(minrelerr_vero_Ac_perm_sub,rwidth=None,color='b', range=(0.0,1.0), bins=vbins, cumulative=False, histtype='bar', align='left', label='opt perm') #,density=True)
plt.legend(loc=7); plt.xlim(-1.e-2,xmax); plt.ylim(0.0,ymax)

plt.show()

In [ ]:
print("np.median(minrelerr_vero_Ac_ps_sub) = ",np.median(minrelerr_vero_Ac_ps_sub[0:]))
print("np.median(minrelerr_vero_Ac_no_perm) = ",np.median(minrelerr_vero_Ac_no_perm[0:]))
print("np.median(minrelerr_vero_Ac_perm_sub) = ",np.median(minrelerr_vero_Ac_perm_sub[0:]))
print(" ")
print("np.median(maxrelerr_vero_Ac_ps_sub) = ",np.median(maxrelerr_vero_Ac_ps_sub[0:]))
print("np.median(maxrelerr_vero_Ac_no_perm) = ",np.median(maxrelerr_vero_Ac_no_perm[0:]))
print("np.median(maxrelerr_vero_Ac_perm_sub) = ",np.median(maxrelerr_vero_Ac_perm_sub[0:]))